In [23]:
### Part 1 ###

def get_input_blocks():
    with open('day16_input.txt') as f:
        return [x.splitlines() for x in f.read().split('\n\n')]

# Input blocks:
# [0]: Fields
# [1]: Your ticket
# [2]: Nearby tickets
input_blocks = get_input_blocks()

# List of Field objects, e.g. depature location etc:
list_of_fields = []

# List of Range objects, e.g. Range(1, 10):
list_of_ranges = []

# Dict taking int to True/False value, depending on if value is acceptable:
num_to_acceptable = {}

# Ticket scanning error rate: the answer we need
ticket_scanning_rate = 0

# Valid ticket indices
valid_ticket_indices = []

class Range():
    def __init__(self, min: int, max: int):
        self.min = min
        self.max = max

class Field():
    def __init__(self, name: str, low_min: int, low_max: int, high_min: int, high_max: int):
        self.name = name
        self.low_min = low_min
        self.low_max = low_max
        self.high_min = high_min
        self.high_max = high_max

def handle_field_block():
    for line in input_blocks[0]:
        split_line_by_space = line.split(' ')
        low_range = list(map(lambda x : int(x), split_line_by_space[-3].split('-')))
        high_range = list(map(lambda x : int(x), split_line_by_space[-1].split('-')))

        split_line_by_colon = line.split(':')
        name_of_field = split_line_by_colon[0]

        list_of_fields.append(Field(name_of_field, low_range[0], low_range[1], high_range[0], high_range[1]))
        list_of_ranges.append(Range(low_range[0], low_range[1]))
        list_of_ranges.append(Range(high_range[0], high_range[1]))

def is_num_acceptable(num: int):
    if num in num_to_acceptable:
        return num_to_acceptable[num]
    
    for range in list_of_ranges:
        if range.min <= num <= range.max:
            num_to_acceptable[num] = True
            return True

    num_to_acceptable[num] = False
    return False

def scan_error_rate_of_ticket(comma_sep_str: str) -> bool:
    error_rate = 0
    nums = list(map(lambda x: int(x), comma_sep_str.split(',')))
    for num in nums:
        if not is_num_acceptable(num):
            error_rate += num
    return error_rate

def is_valid_ticket(comma_sep_str: str) -> bool:
    nums = list(map(lambda x: int(x), comma_sep_str.split(',')))
    for num in nums:
        if num == 0:
            somethig = False
            saer  = True
        if not is_num_acceptable(num):
            return False
    return True

def ticket_scan_error_rate_of_neighbour_tickets():
    error_rate = 0
    for index, ticket_comma_sep_str in enumerate(input_blocks[2]):
        if index == 0:
            continue
        error_rate += scan_error_rate_of_ticket(ticket_comma_sep_str)
        if is_valid_ticket(ticket_comma_sep_str):
            valid_ticket_indices.append(index)
            
    return error_rate

handle_field_block()
ticket_scanning_rate += ticket_scan_error_rate_of_neighbour_tickets()

print(ticket_scanning_rate)

# Shamelessly used in part 2 - cba to implement again
print(valid_ticket_indices)

27850
[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 16, 17, 18, 20, 21, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 79, 80, 81, 83, 84, 85, 89, 91, 93, 94, 96, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 110, 112, 113, 114, 116, 117, 118, 120, 121, 122, 123, 124, 125, 127, 129, 131, 136, 137, 139, 140, 143, 144, 145, 146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 217, 219, 220, 222, 223, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241]


In [33]:
### Part 2 ##

def get_input_blocks():
    with open('day16_input.txt') as f:
        return [x.splitlines() for x in f.read().split('\n\n')]

# Input blocks:
# [0]: Fields
# [1]: Your ticket
# [2]: Nearby tickets
input_blocks = get_input_blocks()

# List of Field objects, e.g. depature location etc:
list_of_fields = []

# To explain the following:
# (field_indices...)[0] = Set of indices (0 corresponding to departure location, 1 to departure station etc) such that that field
#   cannot occupy the first number (so if (field_indices...)[0] contains 0, depature location CANNOT be the first field in the list)
field_indices_that_cannot_exist_at_ticket_index = [set() for x in range(len(input_blocks[0]))]

class Range():
    def __init__(self, min: int, max: int):
        self.min = min
        self.max = max

    def is_in_range(self, num: int):
        return self.min <= num <= self.max

class Field():
    def __init__(self, name: str, low_min: int, low_max: int, high_min: int, high_max: int):
        self.name = name
        self.low_min = low_min
        self.low_max = low_max
        self.high_min = high_min
        self.high_max = high_max
        self.low_range = Range(low_min, low_max)
        self.high_range = Range(high_min, high_max)

def handle_field_block():
    for line in input_blocks[0]:
        split_line_by_space = line.split(' ')
        low_range = list(map(lambda x : int(x), split_line_by_space[-3].split('-')))
        high_range = list(map(lambda x : int(x), split_line_by_space[-1].split('-')))

        split_line_by_colon = line.split(':')
        name_of_field = split_line_by_colon[0]

        list_of_fields.append(Field(name_of_field, low_range[0], low_range[1], high_range[0], high_range[1]))

def scan_ticket(comma_sep_str: str):
    nums = list(map(lambda x : int(x), comma_sep_str.split(',')))
    for ticket_index, num in enumerate(nums):
        for field_index in range(len(input_blocks[0])):
            # We have a ticket index (0 would be first index in the ticket)
            # We have a field index (0 would be referring to departure location field)
            # Let's see if the field_index cannot exist here first:
            if field_index not in field_indices_that_cannot_exist_at_ticket_index[ticket_index]:
                # field_index could potentially live here! If it's out of the range, let's ban it
                if not list_of_fields[field_index].low_range.is_in_range(num) and not list_of_fields[field_index].high_range.is_in_range(num):
                    # This field_index CANNOT be at this ticket_index. Let's add it:
                    field_indices_that_cannot_exist_at_ticket_index[ticket_index].add(field_index)

def handle_nearby_tickets():
    # Only care about valid tickets:
    # Shamelessly grabbed from part 1 (fight me)
    valid_ticket_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 16, 17, 18, 20, 21, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 79, 80, 81, 83, 84, 85, 89, 91, 93, 94, 96, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 110, 112, 113, 114, 116, 117, 118, 120, 121, 122, 123, 124, 125, 127, 129, 131, 136, 137, 139, 140, 143, 144, 145, 146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 217, 219, 220, 222, 223, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241]

    for index in valid_ticket_indices:
        scan_ticket(input_blocks[2][index])


handle_field_block()
handle_nearby_tickets()

# Process of elimination time
field_index_to_ticket_index = {}

# Invert the old thing
field_indices_that_can_exist_at_ticket_index = [set() for x in range(20)]

for ticket_index in range(20):
    for field_index in range(20):
        if field_index not in field_indices_that_cannot_exist_at_ticket_index[ticket_index]:
            field_indices_that_can_exist_at_ticket_index[ticket_index].add(field_index)

while(len(field_index_to_ticket_index) < 20):
    for ticket_index, possible_field_indices in enumerate(field_indices_that_can_exist_at_ticket_index):
        if len(possible_field_indices) == 1:
            field_index = possible_field_indices.pop()
            field_index_to_ticket_index[field_index] = ticket_index
            # Now delete this index from each set in field_indices_that_can_exist_at_ticket_index
            for i in range(20):
                try:
                    field_indices_that_can_exist_at_ticket_index[i].remove(field_index)
                except:
                    pass

print("Field Index to Ticket Index:")
print(field_index_to_ticket_index)

your_ticket = list(map(lambda x : int(x), input_blocks[1][1].split(',')))
answer = 1
for i in range(0, 6):
    answer *= your_ticket[field_index_to_ticket_index[i]]

print(f"Answer!!!: {answer}")


Field Index to Ticket Index:
{12: 9, 15: 12, 10: 4, 17: 15, 18: 6, 11: 17, 2: 11, 0: 13, 5: 7, 1: 10, 3: 2, 4: 3, 7: 16, 9: 5, 16: 0, 8: 1, 19: 19, 13: 14, 6: 18, 14: 8}
Answer!!!: 491924517533
